In [11]:
import pickle, os
import faiss

from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS # intergrates Langchain vectorstore with Faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [4]:
chunk_data_path = 'data/chunks/'
file_name = 'chunks_rec_text_split_cs300_co50_defaultsep.json'

with open(os.path.join(chunk_data_path, file_name), "rb") as fp:
    chunks = pickle.load(fp)

In [12]:
load_dotenv()

True

In [15]:
emb_model = GoogleGenerativeAIEmbeddings(
            model="models/embedding-001",
            google_api_key=os.getenv("GOOGLE_API_KEY")
        ) 

In [24]:
a = emb_model.embed_documents(["d", '3'])

In [37]:
vectors = [emb_model.embed_query(doc.page_content) for doc in chunks]
embedding_mat = np.array(vectors).astype("float16")∑

KeyboardInterrupt: 

In [ ]:
embedding_mat

In [ ]:
embedding_mat.shape[1]

In [34]:
emb_dim = len(emb_model.embed_documents(["test_query"])[0])
quantizer = faiss.IndexFlatIP(emb_dim)
index_ivf = faiss.IndexFlatL2(quantizer, emb_dim, 10)
index_ivf.train(em)
index_ivf.add(embedding_mat)

vector_store = FAISS(
    embedding_function=emb_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id ={}
)
vector_store.add_documents(documents=chunks)

# Delete docs
# vector_store.delete(ids=[])

TypeError: new_IndexFlatL2 expected at most 1 arguments, got 3
Additional information:
Wrong number or type of arguments for overloaded function 'new_IndexFlatL2'.
  Possible C/C++ prototypes are:
    faiss::IndexFlatL2::IndexFlatL2(faiss::idx_t)
    faiss::IndexFlatL2::IndexFlatL2()


In [31]:
results = vector_store.similarity_search(
    query="what is LG energy solution's primary products and research area",
    k=4)

In [32]:
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* information about the Groups reportable segments is as follows: Segment Major products and services LG Energy solution () Automotive batteries, mobile batteries, ESS batteries and others () The Group has determined the reporting segment as a single reporting segment based on the performance [{}]
* Germany December ESS battery sales and others LG Energy Solution (Taiwan) Ltd. 100 100 Taiwan December Mobile battery sales and others Areumnoori Co., Ltd. 100 100 Korea December Facility management and cleaning LG Energy Solution Fund I LLC 100 100 USA December Investment in ventures LG Energy [{}]
* LG Energy Solution Michigan Inc. (1) 100 100 USA December Automotive battery research and manufacturing LG Energy Solution Battery (Nanjing) Co., Ltd. 100 100 China December Automotive battery manufacturing and sales LG Energy Solution Wroclaw sp. z o.o. 100 100 Poland December Automotive battery [{}]
* Ltd. - - - - - 9 9 HAENGBOKNURI CO., LTD. - - - - - 99 99 LEYOU NEW ENERGY MATERIALS(WUXI) 